In [1]:
!pip install -r requirements.txt

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl.metadata (11 kB)
     ---------------------------------------- 0.0/149.4 kB ? eta -:--:--
     -------------------------------------- 149.4/149.4 kB 3.0 MB/s eta 0:00:00
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached pypdf-4.3.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached striprtf-0.0.26-py3-none-any.whl.metadata (2.1 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   --------------- ------------------------ 0.6/1.6 MB 12.6 MB/s eta 0:00:01
   ---------------------------------

DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.68.1 requires pydantic!=1.7,!=1.7.1,!=1.7.2,!=1.7.3,!=1.8,!=1.8.1,<2.0.0,>=1.6.2, but you have pydantic 2.9.2 which is incompatible.


In [5]:
from preprocess_data.parse_data import ParseHandler
from preprocess_data.chunking import ChunkHandler
import os
import io
api_key = os.environ["OPENAI_API_KEY"]

In [2]:
parsehandler = ParseHandler.get_instance(api_key=api_key)

In [3]:
def pdf_to_stream(file_path):
    with open(file_path, 'rb') as pdf_file:
        pdf_content = pdf_file.read()
        pdf_stream = io.BytesIO(pdf_content)
    return pdf_stream

In [4]:
file_path = 'thuvienphapluat/law_2. Tai lieu Khach hang cung cap_5. Salary Regulation 2024 (V) (1).pdf'
pdf_stream = pdf_to_stream(file_path)

In [5]:
image_base64s = parsehandler.pdf_to_images(file_stream=pdf_stream, pdf_path=file_path)

In [6]:
len(image_base64s)

10

In [7]:
def parsing(file_stream, pdf_path):
    parsehandler = ParseHandler.get_instance(api_key=api_key)
    image_base64s = parsehandler.pdf_to_images(file_stream, pdf_path)
    pages, pdf_info, tables, equations = parsehandler.parse_pdf(image_base64s, pdf_path)
    return pages, pdf_info, tables, equations

In [8]:
pages, pdf_info, tables, equations = parsing(pdf_stream, file_path)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [9]:
import json
data = {"pages": pages, "pdf_info": pdf_info, "tables": tables, "equations": equations}
with open("parsing_result/Salary_regulation(V).json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, indent=4, ensure_ascii=False)

In [2]:
import json
with open("parsing_result/Salary_regulation(V).json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

In [11]:
with open(f"parsing_result/tables.md", "a", encoding="utf-8") as file:
    for table in data["tables"]:
        file.write(table["content"] + "\n\n")

In [12]:
with open(f"parsing_result/equations.md", "a", encoding="utf-8") as file:
    for equation in data["equations"]:
        file.write(equation["content"] + "\n\n")

In [3]:
pages, pdf_info, tables, equations = data["pages"], data["pdf_info"], data["tables"], data["equations"]

In [14]:
with open("parsing_result/Salary_regulation(V).md", "w", encoding="utf-8") as file:
    file.write(pdf_info["content"])

In [6]:
chunkhandler = ChunkHandler.get_instance(api_key)

In [7]:
chunk = []

In [8]:
articles = chunkhandler.create_article(pdf_info, pages)
len(articles)

17

In [9]:
forms = chunkhandler.create_form(pdf_info, pages)
len(forms)

0

In [10]:
tbls = chunkhandler.create_table(tables)
len(tbls)

8

In [11]:
eqts = chunkhandler.create_equation(equations)
len(eqts)

8

In [12]:
chunk.extend(articles)
chunk.extend(tbls)
chunk.extend(eqts)
chunk.extend(forms)
len(chunk)

33

In [14]:
with open("parsing_result/Salary_regulation(V)_chunk.json", "w", encoding="utf-8") as json_file:
    json.dump(chunk, json_file, indent=4, ensure_ascii=False)